#**Adidas Sales Analysis ⚽**

**Requirements**
1. **Plotly**

<hr>

#**Business problems that can be solved using this dataset:**
* **Sales Performance Analysis: Which products are excelling in sales, and which are underperforming?**
* **Regional Market Analysis: Which stores are experiencing strong sales, and which ones are lagging?**
* **Profit Margin Analysis: Does the profit margin significantly impact sales?**
* **Efficiency of Sales Methods: Which sales method is more effective - in-store or online?**
* **Price Optimization: Is there a specific price range that achieves better sales than others?**
* **Product Portfolio Optimization I: Determine which products are most profitable, segmented by location.**
* **Market Expansion Opportunities: Assess the best and worst performing stores based on their locations.**
* **Time Series Analysis: Investigate whether there has been a consistent sales trend over time or any noticeable monthly trends.**
* **Predictive Sales Analysis: Develop a forecast for monthly sales.**

<hr>

In [1]:
!pip install xlrd


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install datatable


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install dask


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install seaborn
!pip install statsmodels


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


<hr>

#**Step 1: Importing Libraries**

<hr>

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import datatable as dt  # for a better data uploading
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dask import dataframe as dd  # for better data uploading
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose

<hr>

#**Step 2: Data Loading**

<hr>

**Data Loading using datatable**

In [6]:
import datatable as dt
file_loc = '../AdidasAnalysis/Adidas_US_Sales.csv'
dt_df = dt.fread(file_loc)

In [7]:
dt_df

,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,Foot Locker,1185732,01-01-2020,Northeast,New York,New York,Men's Street Footwear,$50.00,1200,"$6,00,000","$3,00,000",50%,In-store
1,Foot Locker,1185732,02-01-2020,Northeast,New York,New York,Men's Athletic Footwear,$50.00,1000,"$5,00,000","$1,50,000",30%,In-store
2,Foot Locker,1185732,03-01-2020,Northeast,New York,New York,Women's Street Footwear,$40.00,1000,"$4,00,000","$1,40,000",35%,In-store
3,Foot Locker,1185732,04-01-2020,Northeast,New York,New York,Women's Athletic Footwear,$45.00,850,"$3,82,500","$1,33,875",35%,In-store
4,Foot Locker,1185732,05-01-2020,Northeast,New York,New York,Men's Apparel,$60.00,900,"$5,40,000","$1,62,000",30%,In-store
5,Foot Locker,1185732,06-01-2020,Northeast,New York,New York,Women's Apparel,$50.00,1000,"$5,00,000","$1,25,000",25%,In-store
6,Foot Locker,1185732,07-01-2020,Northeast,New York,New York,Men's Street Footwear,$50.00,1250,"$6,25,000","$3,12,500",50%,In-store
7,Foot Locker,1185732,08-01-2020,Northeast,New York,New York,Men's Athletic Footwear,$50.00,900,"$4,50,000","$1,35,000",30%,Outlet
8,Foot Locker,1185732,21-01-2020,Northeast,New York,New York,Women's Street Footwear,$40.00,950,"$3,80,000","$1,33,000",35%,Outlet


Here we are using the datatable for the analysis, since it's faster than normal pandas

In [8]:
dt_df.head()

,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,Foot Locker,1185732,01-01-2020,Northeast,New York,New York,Men's Street Footwear,$50.00,1200,"$6,00,000","$3,00,000",50%,In-store
1,Foot Locker,1185732,02-01-2020,Northeast,New York,New York,Men's Athletic Footwear,$50.00,1000,"$5,00,000","$1,50,000",30%,In-store
2,Foot Locker,1185732,03-01-2020,Northeast,New York,New York,Women's Street Footwear,$40.00,1000,"$4,00,000","$1,40,000",35%,In-store
3,Foot Locker,1185732,04-01-2020,Northeast,New York,New York,Women's Athletic Footwear,$45.00,850,"$3,82,500","$1,33,875",35%,In-store
4,Foot Locker,1185732,05-01-2020,Northeast,New York,New York,Men's Apparel,$60.00,900,"$5,40,000","$1,62,000",30%,In-store
5,Foot Locker,1185732,06-01-2020,Northeast,New York,New York,Women's Apparel,$50.00,1000,"$5,00,000","$1,25,000",25%,In-store
6,Foot Locker,1185732,07-01-2020,Northeast,New York,New York,Men's Street Footwear,$50.00,1250,"$6,25,000","$3,12,500",50%,In-store
7,Foot Locker,1185732,08-01-2020,Northeast,New York,New York,Men's Athletic Footwear,$50.00,900,"$4,50,000","$1,35,000",30%,Outlet
8,Foot Locker,1185732,21-01-2020,Northeast,New York,New York,Women's Street Footwear,$40.00,950,"$3,80,000","$1,33,000",35%,Outlet


In [9]:
# converting the datatable into pandas DataFrame
df = dt_df.to_pandas()

In [10]:
df.shape

(9648, 13)

<hr>

#**Step 3: Exploratory Data Analysis**

<hr>

In [11]:
df.shape

(9648, 13)

In total we are having 9648 sales records, with distribution across the 13 different columns

<hr>

**Data Description**

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9648 entries, 0 to 9647
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Retailer          9648 non-null   object
 1   Retailer ID       9648 non-null   int32 
 2   Invoice Date      9648 non-null   object
 3   Region            9648 non-null   object
 4   State             9648 non-null   object
 5   City              9648 non-null   object
 6   Product           9648 non-null   object
 7   Price per Unit    9648 non-null   object
 8   Units Sold        9648 non-null   int32 
 9   Total Sales       9648 non-null   object
 10  Operating Profit  9648 non-null   object
 11  Operating Margin  9648 non-null   object
 12  Sales Method      9648 non-null   object
dtypes: int32(2), object(11)
memory usage: 904.6+ KB


Insights:
1. Invoice Data is in the format of object, we can change it to the datetime object for better clarity

<hr>

<hr>

**A check for the null values**

In [13]:
df.isnull().sum()

Retailer            0
Retailer ID         0
Invoice Date        0
Region              0
State               0
City                0
Product             0
Price per Unit      0
Units Sold          0
Total Sales         0
Operating Profit    0
Operating Margin    0
Sales Method        0
dtype: int64

No null values detected

<hr>

<hr>

#**Step 4: Data Inspection Phase**


In [14]:
df.head()

,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
0,Foot Locker,1185732,01-01-2020,Northeast,New York,New York,Men's Street Footwear,$50.00,1200,"$6,00,000","$3,00,000",50%,In-store
1,Foot Locker,1185732,02-01-2020,Northeast,New York,New York,Men's Athletic Footwear,$50.00,1000,"$5,00,000","$1,50,000",30%,In-store
2,Foot Locker,1185732,03-01-2020,Northeast,New York,New York,Women's Street Footwear,$40.00,1000,"$4,00,000","$1,40,000",35%,In-store
3,Foot Locker,1185732,04-01-2020,Northeast,New York,New York,Women's Athletic Footwear,$45.00,850,"$3,82,500","$1,33,875",35%,In-store
4,Foot Locker,1185732,05-01-2020,Northeast,New York,New York,Men's Apparel,$60.00,900,"$5,40,000","$1,62,000",30%,In-store


In [15]:
df.tail()

,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
9643,Foot Locker,1185732,24-01-2021,Northeast,New Hampshire,Manchester,Men's Apparel,$50.00,64,"$3,200",$896,28%,Outlet
9644,Foot Locker,1185732,24-01-2021,Northeast,New Hampshire,Manchester,Women's Apparel,$41.00,105,"$4,305","$1,378",32%,Outlet
9645,Foot Locker,1185732,22-02-2021,Northeast,New Hampshire,Manchester,Men's Street Footwear,$41.00,184,"$7,544","$2,791",37%,Outlet
9646,Foot Locker,1185732,22-02-2021,Northeast,New Hampshire,Manchester,Men's Athletic Footwear,$42.00,70,"$2,940","$1,235",42%,Outlet
9647,Foot Locker,1185732,22-02-2021,Northeast,New Hampshire,Manchester,Women's Street Footwear,$29.00,83,"$2,407",$650,27%,Outlet


Insights:
1. Retailer, Region, State, City, Product, Sales Method are having the ordinal category values

2. Retailer ID can be the unique value column

3. Operating Margin can be useful

4. There are some characters in the numerical values, so we need to seperate them as well

<hr>

<hr>

**Working with the insights**

In [16]:
# seperating the extra characters from the numerical columns

df['Total Sales'] = df['Total Sales'].astype(str).str.replace(r'[$, ]', '', regex=True).astype(float)
df['Units Sold'] = df['Units Sold'].astype(str).str.replace(r'[$, ]', '', regex=True).astype(float)
df['Operating Profit'] = df['Operating Profit'].astype(str).str.replace(r'[$, ]', '', regex=True).astype(float)
df['Operating Margin'] = df['Operating Margin'].astype(str).str.replace(r'[%, ]', '', regex=True).astype(float)

In [17]:
df.head()

,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
0,Foot Locker,1185732,01-01-2020,Northeast,New York,New York,Men's Street Footwear,$50.00,1200.0,600000.0,300000.0,50.0,In-store
1,Foot Locker,1185732,02-01-2020,Northeast,New York,New York,Men's Athletic Footwear,$50.00,1000.0,500000.0,150000.0,30.0,In-store
2,Foot Locker,1185732,03-01-2020,Northeast,New York,New York,Women's Street Footwear,$40.00,1000.0,400000.0,140000.0,35.0,In-store
3,Foot Locker,1185732,04-01-2020,Northeast,New York,New York,Women's Athletic Footwear,$45.00,850.0,382500.0,133875.0,35.0,In-store
4,Foot Locker,1185732,05-01-2020,Northeast,New York,New York,Men's Apparel,$60.00,900.0,540000.0,162000.0,30.0,In-store


In [18]:
# converting the date to datetime object
df['Invoice Date'] = pd.to_datetime(df['Invoice Date'], format='%d-%m-%Y')



In [19]:
df.head()

,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
0,Foot Locker,1185732,2020-01-01,Northeast,New York,New York,Men's Street Footwear,$50.00,1200.0,600000.0,300000.0,50.0,In-store
1,Foot Locker,1185732,2020-01-02,Northeast,New York,New York,Men's Athletic Footwear,$50.00,1000.0,500000.0,150000.0,30.0,In-store
2,Foot Locker,1185732,2020-01-03,Northeast,New York,New York,Women's Street Footwear,$40.00,1000.0,400000.0,140000.0,35.0,In-store
3,Foot Locker,1185732,2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,$45.00,850.0,382500.0,133875.0,35.0,In-store
4,Foot Locker,1185732,2020-01-05,Northeast,New York,New York,Men's Apparel,$60.00,900.0,540000.0,162000.0,30.0,In-store


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9648 entries, 0 to 9647
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Retailer          9648 non-null   object        
 1   Retailer ID       9648 non-null   int32         
 2   Invoice Date      9648 non-null   datetime64[ns]
 3   Region            9648 non-null   object        
 4   State             9648 non-null   object        
 5   City              9648 non-null   object        
 6   Product           9648 non-null   object        
 7   Price per Unit    9648 non-null   object        
 8   Units Sold        9648 non-null   float64       
 9   Total Sales       9648 non-null   float64       
 10  Operating Profit  9648 non-null   float64       
 11  Operating Margin  9648 non-null   float64       
 12  Sales Method      9648 non-null   object        
dtypes: datetime64[ns](1), float64(4), int32(1), object(7)
memory usage: 942.3+ KB


<hr>

#**BP1: Sales Performance Analysis**

**Which product are excelling in sales, and which are underperforming?**

In [21]:
# finding the total revenue
total_revenue = df['Total Sales'].sum()
print(f'The total Revenue: {total_revenue}')

The total Revenue: 899902125.0


In [22]:
# Aggregating the total sales and units sold for each product
prod_sales = df.groupby('Product').agg({'Total Sales':'sum', 'Units Sold':'sum'}).reset_index()
# sorting product by the total values by decreasing to find the top prod
top_prod = prod_sales.sort_values(by='Total Sales', ascending = False)
# results
print('The top performing Products in the dataset')
top_prod

The top performing Products in the dataset


,Product,Total Sales,Units Sold
2,Men's Street Footwear,208826244.0,593320.0
3,Women's Apparel,179038860.0,433827.0
1,Men's Athletic Footwear,153673680.0,435526.0
5,Women's Street Footwear,128002813.0,392269.0
0,Men's Apparel,123728632.0,306683.0
4,Women's Athletic Footwear,106631896.0,317236.0


Insights:

* The **Men's Street Footwear** are the most selling item in the entire dataset, with total sales value of 208826244

In [23]:
# reversing the list to find the least performing product
least_prod = prod_sales.sort_values(by = 'Total Sales')
print('Least Performed')
least_prod

Least Performed


,Product,Total Sales,Units Sold
4,Women's Athletic Footwear,106631896.0,317236.0
0,Men's Apparel,123728632.0,306683.0
5,Women's Street Footwear,128002813.0,392269.0
1,Men's Athletic Footwear,153673680.0,435526.0
3,Women's Apparel,179038860.0,433827.0
2,Men's Street Footwear,208826244.0,593320.0


Insights:

* From here we can clearly see that the **Women's Athletic Footwear** is the one which having the lowest sales in all, with values of 106631896

<hr>

<hr>

#**BP2: Regional Market Analysis**

**Which stores are experiencing strong sales, and which ones are lagging**

In [24]:
df.head()

,Retailer,Retailer ID,Invoice Date,Region,State,City,Product,Price per Unit,Units Sold,Total Sales,Operating Profit,Operating Margin,Sales Method
0,Foot Locker,1185732,2020-01-01,Northeast,New York,New York,Men's Street Footwear,$50.00,1200.0,600000.0,300000.0,50.0,In-store
1,Foot Locker,1185732,2020-01-02,Northeast,New York,New York,Men's Athletic Footwear,$50.00,1000.0,500000.0,150000.0,30.0,In-store
2,Foot Locker,1185732,2020-01-03,Northeast,New York,New York,Women's Street Footwear,$40.00,1000.0,400000.0,140000.0,35.0,In-store
3,Foot Locker,1185732,2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,$45.00,850.0,382500.0,133875.0,35.0,In-store
4,Foot Locker,1185732,2020-01-05,Northeast,New York,New York,Men's Apparel,$60.00,900.0,540000.0,162000.0,30.0,In-store


For this we need to work with the cities to find out the best and the worst performing stores

In [25]:
# cities that are unique
unique_cities = df['City'].unique()
print(f'The unique cities: {unique_cities}')

The unique cities: ['New York' 'Houston' 'San Francisco' 'Los Angeles' 'Chicago' 'Dallas'
 'Philadelphia' 'Las Vegas' 'Denver' 'Seattle' 'Miami' 'Minneapolis'
 'Billings' 'Knoxville' 'Omaha' 'Birmingham' 'Portland' 'Anchorage'
 'Honolulu' 'Orlando' 'Albany' 'Cheyenne' 'Richmond' 'Detroit' 'St. Louis'
 'Salt Lake City' 'New Orleans' 'Boise' 'Phoenix' 'Albuquerque' 'Atlanta'
 'Charleston' 'Charlotte' 'Columbus' 'Louisville' 'Jackson' 'Little Rock'
 'Oklahoma City' 'Wichita' 'Sioux Falls' 'Fargo' 'Des Moines' 'Milwaukee'
 'Indianapolis' 'Baltimore' 'Wilmington' 'Newark' 'Hartford' 'Providence'
 'Boston' 'Burlington' 'Manchester']


In [26]:
# how many are there
noOfCities = df['City'].nunique()
print(f'The total number of unique cities: {noOfCities}')

The total number of unique cities: 52


In [27]:
# grouping the cities and retailers and summing up the units sold
unitsCities = df.groupby(['City', 'Retailer'])['Units Sold'].sum().reset_index()

# sorting the results withing the city to find the top and worst store
sortedDf = unitsCities.sort_values(by = ['City', 'Units Sold'], ascending = [True, False])

# getting the performers
topStore = sortedDf.groupby('City').head(1)

# worst
leastStore = sortedDf.groupby('City').tail(1)

In [28]:
# results
print('Top Performing Store in Each City')
topStore.head()

Top Performing Store in Each City


,City,Retailer,Units Sold
1,Albany,West Gear,47133.0
2,Albuquerque,Kohl's,43752.0
4,Anchorage,Amazon,26749.0
7,Atlanta,Sports Direct,41414.0
8,Baltimore,Foot Locker,9322.0


In [29]:
# least performing cities
print('Least Performing Store in city')
leastStore.head()

Least Performing Store in city


,City,Retailer,Units Sold
0,Albany,Kohl's,10053.0
3,Albuquerque,Sports Direct,8881.0
5,Anchorage,Foot Locker,4066.0
6,Atlanta,Foot Locker,14977.0
10,Baltimore,West Gear,5647.0


The list of all the store that are least performing in each city

<hr>

<hr>

#**Profit Margin Analysis**

**Does the operating profit significantly impact sales**

In [30]:
def corrnFunc(val):
  if val >=0.90:
    return f'The value of correlation is {val}, that shows high correlation between both entities'
  elif val >=0.70 and val<0.90:
    return f'The values of correlation is {val}, that should a positive correlation that is not that bad'
  else:
    return 'Weak correlation'

In [31]:
# checking the correlation between Operating profit and total sales
corrn = df['Operating Profit'].corr(df['Total Sales'])
corrnFunc(corrn)

'The value of correlation is 0.9563074748281607, that shows high correlation between both entities'

In [32]:
# checking the correlation between Operating profit and Units Sold
corrn = df['Operating Profit'].corr(df['Units Sold'])
corrnFunc(corrn)

'The values of correlation is 0.8923794236295572, that should a positive correlation that is not that bad'

Insights:

* The correlation for the both the comparison are high.

* This indicates that if the total sales will go up then the Operating Profit will also go up.

* **So the answer is Yes, the operating Profit does affect Units Sold in a positive way**

<hr>

<hr>

#**BP4: Efficiency of Sales Methods**

**Which Sales method is more effective --> In-store or online**

In [33]:
# finding the unique cols in df
salesMeth = df["Sales Method"].unique()
print(f'The available sales methods: {salesMeth}')

The available sales methods: ['In-store' 'Outlet' 'Online']


In [34]:
# using plotly we will try to plot these all
!pip install us

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/208.0 kB ? eta -:--:--
     -------------------------------------- 208.0/208.0 kB 6.4 MB/s eta 0:00:00
  Created wheel for us: filename=us-3.1.1-py3-none-any.whl size=12607 sha256=9006f08a56e147f6e80d5cd7ce3e911301f178766dc04419d1d61501b38b3bd7
  Stored in directory: c:\users\aman raghuvanshi\appdata\local\pip\cache\wheels\88\1d\36\4c39b41019d5c316628db111f85de00af4476af5b9c5ddccc3
Successfully built us



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
from us import states
stateCol = df['State']

# Getting the two letter short names of the state names
stateName = []
for x in stateCol:
  try:
    # finding the name for the states
    name = states.lookup(x).abbr
  except AttributeError:
    # incase we are not able to find the same, we set the Name to Unknown / None
    name = 'Unknown'
  stateName.append(name)


In [36]:
# creating a new column having the shortforms
df['State_ShortForm'] = stateName
df['State_ShortForm'].head()

0    NY
1    NY
2    NY
3    NY
4    NY
Name: State_ShortForm, dtype: object

In [37]:
df['State_ShortForm'].unique()

array(['NY', 'TX', 'CA', 'IL', 'PA', 'NV', 'CO', 'WA', 'FL', 'MN', 'MT',
       'TN', 'NE', 'AL', 'ME', 'AK', 'HI', 'WY', 'VA', 'MI', 'MO', 'UT',
       'OR', 'LA', 'ID', 'AZ', 'NM', 'GA', 'SC', 'NC', 'OH', 'KY', 'MS',
       'AR', 'OK', 'KS', 'SD', 'ND', 'IA', 'WI', 'IN', 'WV', 'MD', 'DE',
       'NJ', 'CT', 'RI', 'MA', 'VT', 'NH'], dtype=object)

In [41]:
!pip install --upgrade nbformat
!pip install --upgrade plotly




[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


**Using the choropleth Map trying to plot Sales by each state, product and sales Method in US**

In [42]:
df['Total Sales'] = df['Total Sales'].replace('[$,]', "", regex = True).astype(float)

# making canvas
fig = go.Figure()
#all the unique products and sales method
prod = df['Product'].unique()
salesMeth = df['Sales Method'].unique()

# adding the traces for each combination of products and sales
for x in prod:
  for y in salesMeth:
    filterDf = df[(df['Product']==x) & (df['Sales Method']==y)]
    stateSales = filterDf.groupby('State_ShortForm')['Total Sales'].sum().reset_index()

    fig.add_trace(
        go.Choropleth(
            locations = stateSales['State_ShortForm'],
            z = stateSales['Total Sales'],
            locationmode = 'USA-states',
            colorscale = 'tealrose',
            name = f'{x} - {y}',
            showscale = True,
            visible = False, # hidding all traces initially
        )
    )


# updating the layout with dropdowns (this is for products)
prod_button = [
    {'label': x,
     'method':'update',
     'args':[{'visible':[traces.name.startswith(x) for traces in fig.data]}]}
    for x in prod
]

# updating the same for sales
sales_button = [
    {'label': y,
     'method':'update',
     'args':[{'visible':[y in trace.name for trace in fig.data]}]}
    for y in salesMeth
]

fig.update_layout(
    updatemenus = [
        {'buttons':prod_button,
         'direction':'down',
         'showactive': True,
         'x':0.25,
         'xanchor':'left',
         'y':1.15,
         'yanchor':'top'
         },
        {'buttons':sales_button,
         'direction':'down',
         'showactive': True,
         'x':0.75,
         'xanchor':'left',
         'y':1.15,
         'yanchor':'top'
         }],
    geo = dict(scope = 'usa'),
    title = 'Total Sales by State, Product and Sales Method available'
)

# displaying the first product and first sales
if fig.data:
  fig.data[0].visible = True

fig.show()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

'aggrnyl', 'agsunset', 'algae', 'amp', 'armyrose', 'balance',
             'blackbody', 'bluered', 'blues', 'blugrn', 'bluyl', 'brbg',
             'brwnyl', 'bugn', 'bupu', 'burg', 'burgyl', 'cividis', 'curl',
             'darkmint', 'deep', 'delta', 'dense', 'earth', 'edge', 'electric',
             'emrld', 'fall', 'geyser', 'gnbu', 'gray', 'greens', 'greys',
             'haline', 'hot', 'hsv', 'ice', 'icefire', 'inferno', 'jet',
             'magenta', 'magma', 'matter', 'mint', 'mrybm', 'mygbm', 'oranges',
             'orrd', 'oryel', 'oxy', 'peach', 'phase', 'picnic', 'pinkyl',
             'piyg', 'plasma', 'plotly3', 'portland', 'prgn', 'pubu', 'pubugn',
             'puor', 'purd', 'purp', 'purples', 'purpor', 'rainbow', 'rdbu',
             'rdgy', 'rdpu', 'rdylbu', 'rdylgn', 'redor', 'reds', 'solar',
             'spectral', 'speed', 'sunset', 'sunsetdark', 'teal', 'tealgrn',
             'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid',
             'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr',
             'ylorrd'

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9648 entries, 0 to 9647
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Retailer          9648 non-null   object        
 1   Retailer ID       9648 non-null   int32         
 2   Invoice Date      9648 non-null   datetime64[ns]
 3   Region            9648 non-null   object        
 4   State             9648 non-null   object        
 5   City              9648 non-null   object        
 6   Product           9648 non-null   object        
 7   Price per Unit    9648 non-null   object        
 8   Units Sold        9648 non-null   float64       
 9   Total Sales       9648 non-null   float64       
 10  Operating Profit  9648 non-null   float64       
 11  Operating Margin  9648 non-null   float64       
 12  Sales Method      9648 non-null   object        
 13  State_ShortForm   9648 non-null   object        
dtypes: datetime64[ns](1), fl

In [ ]:
# working on the method to segment and aggregate data by Sales
salesData = df.groupby('Sales Method').agg({'Total Sales': 'sum', 'Operating Profit': 'sum'})

# Calculating the Operating Margin for each sales method
salesData['Operating Margin'] = salesData['Operating Profit'] / salesData['Total Sales']

salesData

,Total Sales,Operating Profit,Operating Margin
Sales Method,,,
In-store,356643750.0,127591382.0,0.357756
Online,247672882.0,96555337.0,0.389850
Outlet,295585493.0,107988403.0,0.365337


In [ ]:
#Which method is best
# Segmenting and aggregating data by 'Sales Method'
sales_method_grouped = data.groupby('Sales Method').agg({'Total Sales': 'sum', 'Operating Profit': 'sum'})

# Calculating the Operating Margin for each sales method
sales_method_grouped['Operating Margin'] = sales_method_grouped['Operating Profit'] / sales_method_grouped['Total Sales']

sales_method_grouped